# Import Libraries

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import string
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from models.train_classifier import *

# Load the Data

In [ ]:
x, y = load_data(
        'sqlite:///data/02_stg//stg_disaster_response.db',
        'stg_disaster_response',
        FEATURE_COLUMNS, 
        TARGET_COLUMNS
    )
print(x.shape)
print(y.shape)

# Process the data

In [ ]:
x = tokenize(x)
print(x[0])

# Train the Pipeline